# MA556 Group Project

## Part 1: Attain Survey Weights

In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.special import expit # Logistic function

# Generate values of z from the normal distribution model
def generate_z(mu_z, sigma_z, N):
    z = np.random.normal(mu_z, sigma_z, N)
    return z

# Calculate the propensity score using logistic regression
def compute_propensity_scores(data, psi, predictors=['Intercept', 'Age', 'Sex', 'Z']):
    # Calculate v'ψ
    linear_predictor = np.dot(data[predictors], psi)
    
    # Apply the logistic to propensity
    propensity_scores = expit(linear_predictor)

    return propensity_scores

def compute_survey_weights(data, propensity_scores):
    N = len(data)  # Total # individuals
    f = 1 / N  # Sample fraction

    # Calculate W_i
    original_weights = N / propensity_scores

    # Calculate adjusted w_i
    adjusted_weights = f * original_weights / original_weights.mean()

    # Calculate effective sample size n_hat
    n_hat = (original_weights.sum())**2 / (original_weights**2).sum()

    # Calculate final adjusted survey weights
    final_adjusted_weights = n_hat * original_weights / original_weights.sum()

    return final_adjusted_weights


# Import data
data = pd.read_csv('data_table.csv')

# Generate values of z
mu_z = 0
sigma_z = 1

N = len(data)
data['Z'] = generate_z(mu_z, sigma_z, N)

# Fit the logistic regression
predictors = ['Intercept', 'Age', 'Sex', 'Z']
response = 'Treatment'

model = sm.Logit(data[response], data[predictors])
results = model.fit()

# The fitted coefficients 'psi'
psi = results.params

# Fit logistic regression
X = data[predictors]
y = data[response]

propensity_scores = compute_propensity_scores(data, psi)
adjusted_weights = compute_survey_weights(data, propensity_scores)

data['AdjustedWeight'] = adjusted_weights

print(data['AdjustedWeight'])


Optimization terminated successfully.
         Current function value: 0.609505
         Iterations 5
0     1.320812
1     1.473084
2     1.096989
3     1.149462
4     1.061786
5     0.973768
6     0.944140
7     0.730460
8     0.844241
9     1.136599
10    0.704500
11    0.741415
12    0.869948
13    0.675020
14    1.869590
15    1.613470
16    0.808174
17    1.091557
18    0.687063
19    1.176259
20    0.641657
21    0.642570
22    1.040428
23    0.932089
24    0.772594
25    0.747105
26    0.697096
27    0.735643
28    0.769297
29    0.625941
30    0.614848
31    0.678257
32    0.662432
33    0.662491
34    0.977431
35    0.694466
36    0.625606
Name: AdjustedWeight, dtype: float64


## Depricated Code

In [ ]:
# df = pd.read_csv('data.csv')
# df['Intercept'] = 1
# df['Treatment'] = df['Subject'].apply(lambda x: 0 if 'C' in x else 1)
# df = pd.get_dummies(df, columns=['Sex'], drop_first=True)
# df = df.rename(columns={'Sex_M':'Sex'})

# df.to_csv('data_table.csv')
